In [12]:
import numpy as np
import cv2
from copy import deepcopy
from PIL import Image
import pytesseract 
import sys
import urllib




def process(img):
  #cv2.imshow("Input",img)
  imgBlurred = cv2.GaussianBlur(img, (5,5), 0)
  gray = cv2.cvtColor(imgBlurred, cv2.COLOR_BGR2GRAY)

  sobelx = cv2.Sobel(gray,cv2.CV_8U,1,0,ksize=3)
  #cv2.imshow("Sobel",sobelx)
  #cv2.waitKey(0)
  ret2,threshold_img = cv2.threshold(sobelx,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
  #cv2.imshow("Threshold",threshold_img)
  #cv2.waitKey(0)
  return threshold_img

def choose_largest(plate):
  print ("CLEANING PLATE. . .")
  gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
  #kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
  #thresh= cv2.dilate(gray, kernel, iterations=1)

  _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
  im1,contours,hierarchy = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

  if contours:
    areas = [cv2.contourArea(c) for c in contours]
    max_index = np.argmax(areas)

    max_cnt = contours[max_index]
    max_cntArea = areas[max_index]
    x,y,w,h = cv2.boundingRect(max_cnt)
    if not checkRatio(max_cntArea,w,h):
      return plate,None

    cleaned_final = thresh[y:y+h, x:x+w]
    #cv2.imshow("Function Test",cleaned_final)
    return cleaned_final,[x,y,w,h]

  else:
    return plate,None


def get_edges(threshold_img):
  element = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(17, 3))
  morph_img_threshold = threshold_img.copy()
  cv2.morphologyEx(src=threshold_img, op=cv2.MORPH_CLOSE, kernel=element, dst=morph_img_threshold)
  #cv2.imshow("Morphed",morph_img_threshold)
  cv2.waitKey(0)

  im2,contours, hierarchy= cv2.findContours(morph_img_threshold,mode=cv2.RETR_EXTERNAL,method=cv2.CHAIN_APPROX_NONE)
  return contours


def checkRatio(area, width, height):
  ratio = float(width) / float(height)
  if ratio < 1:
    ratio = 1 / ratio
    
  aspect = 4.7272
  min = 15*aspect*15  # mini area
  max = 125*aspect*125  # max area

  rmin = 3
  rmax = 6

  if (area < min or area > max) or (ratio < rmin or ratio > rmax):
    return False
  return True

def checkBlankness(plate):
  avg = np.mean(plate)
  if(avg>=115):
    return True
  else:
    return False

def validation_check(rect):
  (x, y), (width, height), rect_angle = rect
  if(width>height):
    angle = -rect_angle
  else:
    angle = 90 + rect_angle

  if angle>15:
    return False

  if height == 0 or width == 0:
    return False

  area = height*width
  if not checkRatio(area,width,height):
    return False
  else:
    return True
 



def choose_plate(img,contours):
  #count=0
  for i,cnt in enumerate(contours):
    min_rect = cv2.minAreaRect(cnt)
    if validation_check(min_rect):

      x,y,w,h = cv2.boundingRect(cnt)
      plate_img = img[y:y+h,x:x+w]

      if(checkBlankness(plate_img)):
        #count+=1
        clean_plate, rect = choose_largest(plate_img)
        if rect:
          x1,y1,w1,h1 = rect
          x,y,w,h = x+x1,y+y1,w1,h1
          #cv2.imshow("Cleaned Plate",clean_plate)
          cv2.waitKey(0)
          img = cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
          area=(x,y,(x+w),(y+h))
          get_chars(img,area)
          break

def get_chars(img,area):
  cv2.imwrite("drive/My Drive/Colab/testp.jpg",img) 
  img1=Image.open("drive/My Drive/Colab/testp.jpg")
  img1=img1.crop(area)
  img1.save("drive/My Drive/Colab/testf.jpg")

  text = pytesseract.image_to_string(Image.open("drive/My Drive/Colab/testf.jpg"), lang='eng')
  if(len(text)<5):
    gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    cv2.imwrite("drive/My Drive/Colab/testf.jpg", gray)
    text = pytesseract.image_to_string(Image.open("drive/My Drive/Colab/testf.jpg"), lang='eng')

  print("Detected Text : ",text)
       
          #cv2.waitKey(0)
  
	



if __name__ == '__main__':
  print ("DETECTING PLATE . . .")

  #img = cv2.imread("testData/Final.JPG")
  img = cv2.imread("drive/My Drive/Colab/test1.jpg")
  threshold_img = process(img)
  contours= get_edges(threshold_img)



  choose_plate(img,contours)

DETECTING PLATE . . .
CLEANING PLATE. . .
Detected Text :  TH07BU54:27


In [0]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [9]:
!pip install pyrebase

In [10]:
!apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [11]:
!pip install pytesseract